In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [26]:
events = pd.read_csv("dataset_events.csv")
print(events.shape)
events.head()

(653419, 4)


,customer_id,product_id,type,timestamp
0,1,19685,view_product,1527812004
1,1,19685,view_product,1527812041
2,1,19685,add_to_cart,1527812046
3,1,19685,view_product,1527812048
4,1,19685,view_product,1527812050


In [27]:

events.loc[pd.to_datetime(events['timestamp']).index, 'date'] = pd.to_datetime(events['timestamp']).values
events['date'] = events['date'].apply(lambda x: str(x).split()[0])
events['date'].value_counts(), events.shape

(1970-01-01    653419
 Name: date, dtype: int64, (653419, 5))

In [28]:
purchases = events[events.type == 'purchase_item'].drop(columns=['timestamp', 'date'])
carts = events[events.type == 'add_to_cart'].drop(columns=['timestamp', 'date'])
views = events[events.type == 'view_product'].drop(columns=['timestamp', 'date'])

purchases.head()

,customer_id,product_id,type
161,44,24129,purchase_item
162,44,3814,purchase_item
360,121,16959,purchase_item
396,132,3536,purchase_item
649,234,26045,purchase_item


In [29]:
purchases_with_no_duplicates = purchases.drop_duplicates(['product_id', 'customer_id'])
add_cart_with_no_duplicates = carts.drop_duplicates(['product_id', 'customer_id'])
views_with_no_duplicates = views.drop_duplicates(['product_id', 'customer_id'])

purchases_with_no_duplicates.head()

,customer_id,product_id,type
161,44,24129,purchase_item
162,44,3814,purchase_item
360,121,16959,purchase_item
396,132,3536,purchase_item
649,234,26045,purchase_item


In [38]:
grouped_purchases_count_by_product_id = purchases_with_no_duplicates.groupby(['product_id']).customer_id.agg('count').to_frame('n_unique_purchases').reset_index()
sorted_popular_items_by_purchase = grouped_purchases_count_by_product_id.sort_values('n_unique_purchases', ascending=False)

grouped_carts_count_by_product_id = add_cart_with_no_duplicates.groupby(['product_id']).customer_id.agg('count').to_frame('n_unique_cart_adds').reset_index()
sorted_popular_items_by_cart_add = grouped_carts_count_by_product_id.sort_values('n_unique_cart_adds', ascending=False)

grouped_views_count_by_product_id = views_with_no_duplicates.groupby(['product_id']).customer_id.agg('count').to_frame('n_unique_views').reset_index()
sorted_popular_items_by_views = grouped_views_count_by_product_id.sort_values('n_unique_views', ascending=False)

sorted_popular_items_by_views.head()

,product_id,n_unique_views
22024,22031,2052
11215,11219,1884
25009,25016,1832
21115,21122,1621
20584,20591,1473


In [33]:
sorted_popular_items_by_views.product_id.values

array([22031, 11219, 25016, ...,  5599,  5600, 28369], dtype=int64)

In [34]:
np.save("sorted_popular_items_by_purchase", sorted_popular_items_by_purchase.product_id.values)
np.save("solrted_popular_items_by_cart_add", sorted_popular_items_by_cart_add.product_id.values)
np.save("sorted_popular_items_by_views", sorted_popular_items_by_views.product_id.values)